# **Google Maps Timeline to Fog of World Converter**

This script converts the exported **Google Maps Timeline** data (*location-history.json*) into a **GPX file of location points** to be imported to **Fog of World**.

## **Why Points:**

All data from the Google Maps Timeline location history, including paths, **will be extracted to points** due to the inaccuracy of the timeline paths: the paths simply connect the recorded points, so the lines often jump across buildings instead of being drawn along the roads.

**To restore paths:** Connect points manually in the app: *Settings → Database → Track Editor* (*or long-pressing the map menu icon*). Optional: Use the generated interactive map ***timeline-map.html*** as a reference.

## **Original Data:**
Google Maps Timeline data has four categories:

| Category | Geo Data | For |
| :-: | --- | --- |
| ***visit*** | point | places visited |
| ***activity*** | start point + end point | activities like *walking*, *cycling*, *in subway*, etc., from one point to another |
| ***timelinePath*** | collection of all recorded points along the path | movements paths |
| ***timelineMemory*†** | n/a | records for distant destinations (?) |


† *timelineMemory* will be discarded

## **Output Files:**
1. ***timeline.gpx*** → GPX tracks†† to be imported to Fog of World
2. ***timeline.csv*** → CSV table of all extracted points with time††† (in UTC), coordinates, category, and notes†††
3. ***timeline-map.html*** → an interactive Plotly map

†† Fog of World can't read points from the GPX file. Each point will add a small offset (1e-6) to form a short track for usability by FoW.

††† *time* and  *notes* vary by category:

| Category | timeUTC | notes | notes exapmle |
| :-: | --- | --- | --- |
| ***visit*** | start time | calculated duration of the visit (end time - start time) | duration: 0.73 h |
| ***activity*** | start time if is the start point; end time if is the end point | is start or end point | start |
| ***timelinePath*** | calculated timestamp of this point | minutes offset from path start time (in whole hours) | 2013-05-03 18:00 + 74' |

# **Before Runing the Script:**
### Replace the sample *location-history.json* file with your own. Make sure the *location-history.json* file and this *.ipynb* file are in the same directory.
### If Using Google Colab:

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# # cd to your directory
# %cd /content/drive/MyDrive/Personal projects/Google Maps Timeline to FoW/script-sampleData

# **Script:**

In [ ]:
# Install required packages
%pip install pandas plotly gpxpy

In [3]:
import json
import pandas as pd
import plotly.express as px
import gpxpy
import os


# -------- Load & base DF --------
items = json.load(open("location-history.json"))
timeline_df = (
    pd.DataFrame([
        {
            "startTime": it.get("startTime"),
            "endTime":   it.get("endTime"),
            "category":  (ck := next((k for k in it if k not in ("startTime","endTime")), None)),
            "data":      it.get(ck),
        } for it in items
    ])
    .assign(
        startTime=lambda d: pd.to_datetime(d["startTime"], errors="coerce", utc=True).dt.tz_localize(None),
        endTime=lambda d: pd.to_datetime(d["endTime"],   errors="coerce", utc=True).dt.tz_localize(None),
    )
    .sort_values("startTime", na_position="last").reset_index(drop=True)
)

parts = {k: g.reset_index(drop=True) for k, g in timeline_df.groupby("category", sort=False)}
# Robust defaults if a category is missing
visit        = parts.get("visit",        pd.DataFrame(columns=["startTime","endTime","category","data"]))
activity     = parts.get("activity",     pd.DataFrame(columns=["startTime","endTime","category","data"]))
timelinePath = parts.get("timelinePath", pd.DataFrame(columns=["startTime","endTime","category","data"]))


# -------- Helper --------
def geo_to_xy(s: pd.Series, prefix=""):
    return (s.str.extract(r"^geo:\s*(?P<lat>[-+]?\d+(?:\.\d+)?),\s*(?P<lon>[-+]?\d+(?:\.\d+)?)$")
             .apply(pd.to_numeric, errors="coerce").add_prefix(prefix))


# -------- Part 1: visit --------
if not visit.empty:
    timeline_visits = (
        visit.assign(
            place=lambda d: d["data"].map(lambda x: (x or {}).get("topCandidate", {}).get("placeLocation"))
        )
        .pipe(lambda d: d.join(geo_to_xy(d["place"])))
        .assign(
            duration_h=lambda d: (d["endTime"] - d["startTime"]).dt.total_seconds()/3600,
            notes=lambda d: "duration: " + d["duration_h"].round(2).astype(str) + " h",
        )
        .rename(columns={"startTime":"timeUTC"})
        [["timeUTC","lat","lon","category","notes"]]
    )
else:
    timeline_visits = pd.DataFrame(columns=["timeUTC","lat","lon","category","notes"])

# -------- Part 2: activity --------
if not activity.empty:
    activity = (
        activity.assign(
            start=lambda d: d["data"].map(lambda x: (x or {}).get("start")),
            end=lambda d:   d["data"].map(lambda x: (x or {}).get("end")),
        )
        .pipe(lambda d: d.join(geo_to_xy(d["start"], "start_")).join(geo_to_xy(d["end"], "end_")))
    )

    timeline_activity = (
        pd.concat([
            activity.rename(columns={"startTime":"timeUTC","start_lat":"lat","start_lon":"lon"})
                    [["timeUTC","lat","lon","category"]].assign(notes="start"),
            activity.rename(columns={"endTime":"timeUTC","end_lat":"lat","end_lon":"lon"})
                    [["timeUTC","lat","lon","category"]].assign(notes="end"),
        ], ignore_index=True)
        .dropna(subset=["timeUTC"]).sort_values("timeUTC").reset_index(drop=True)
    )
else:
    timeline_activity = pd.DataFrame(columns=["timeUTC","lat","lon","category","notes"])

# -------- Part 3: timelinePath (points) --------
if not timelinePath.empty:
    tp = timelinePath.explode("data").dropna(subset=["data"]).reset_index(drop=True)
    if not tp.empty:
        tp = tp.join(pd.json_normalize(tp.pop("data")))
        tp = tp.join(geo_to_xy(tp["point"]))
        tp = tp.assign(
            timeUTC=lambda d: d["startTime"] + pd.to_timedelta(
                pd.to_numeric(d["durationMinutesOffsetFromStartTime"], errors="coerce"), unit="m"),
            notes=lambda d: d["startTime"].astype(str).str[:-3] + " + " +
                            d["durationMinutesOffsetFromStartTime"].astype(str) + "'",
        )
        timeline_path_pts = tp.loc[:, ["timeUTC","lat","lon","category","notes"]] \
                              .dropna(subset=["timeUTC","lat","lon"]).reset_index(drop=True)
    else:
        timeline_path_pts = pd.DataFrame(columns=["timeUTC","lat","lon","category","notes"])
else:
    timeline_path_pts = pd.DataFrame(columns=["timeUTC","lat","lon","category","notes"])

# -------- Combine --------
timeline = (
    pd.concat([timeline_visits, timeline_activity, timeline_path_pts], ignore_index=True)
      .sort_values("timeUTC")
      .reset_index(drop=True)
)


# -------- Export --------
if timeline.empty:
    print("No timeline data to plot/export (missing categories or no valid coordinates).")
else:
    # Create the output directory
    output_dir = "output"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # GPX export (single track, two points per row)
    gpx = gpxpy.gpx.GPX()
    track = gpxpy.gpx.GPXTrack(); gpx.tracks.append(track)

    offset = 1e-6
    for _, r in timeline.dropna(subset=["lat","lon","timeUTC"]).iterrows():
        # Create a new segment for each point pair
        seg = gpxpy.gpx.GPXTrackSegment(); track.segments.append(seg)
        seg.points.append(gpxpy.gpx.GPXTrackPoint(r["lat"], r["lon"], time=r["timeUTC"]))
        seg.points.append(gpxpy.gpx.GPXTrackPoint(r["lat"]+offset, r["lon"]+offset, time=r["timeUTC"]))

    with open("output/timeline.gpx","w") as f:
        f.write(gpx.to_xml())
    print("GPX file timeline.gpx saved")


    # CSV export
    timeline.to_csv("output/timeline.csv", index=False)
    print("CSV file timeline.csv saved")

    # Map & export
    fig = px.scatter_map(
        timeline,
        lat="lat", lon="lon",
        color="category",
        hover_name="category",
        hover_data={"category": False, "timeUTC": True, "lat": False, "lon": False, "notes": True},
        zoom=1.23, height=600, width=1200,
        color_discrete_map={"visit":"#EF553B","timelinePath":"#AB63FA","activity":"#00CC96"},
    )

    fig.update_layout(
        map_style="carto-voyager",
        margin=dict(r=0,t=0,l=0,b=0),
        legend=dict(orientation="v", x=0.004, y=0.01, xanchor="left", yanchor="bottom",
                    bgcolor="rgba(255,255,255,1)", valign="top", grouptitlefont=dict(size=14), font=dict(size=12)),
        updatemenus=[dict(
            direction="down", x=0.004, y=0.99, xanchor="left", yanchor="top",
            buttons=[
                {"label":"Default Basemap",       "method":"relayout", "args":[{"map.style":"carto-voyager"}]},
                {"label":"Default (no labels)",   "method":"relayout", "args":[{"map.style":"carto-voyager-nolabels"}]},
                {"label":"Satellite Basemap",     "method":"relayout", "args":[{"map.style":"satellite-streets"}]},
                {"label":"Satellite (no labels)", "method":"relayout", "args":[{"map.style":"satellite"}]},
            ]
        )]
    )

    fig.write_html("output/timeline-map.html")
    print("Map file timeline-map.html saved")

    fig.show()

GPX file timeline.gpx saved
CSV file timeline.csv saved
Map file timeline-map.html saved
